# Climate Change impact for River Leven at Newby Bridge using HBV model in eWaterCycle
This is a copy of [this notebook](../example_model_run_HBV.ipynb) that showed how to run the HBV model for the river Leven using ERA5 data as forcing.

In this notebook we show how to change the forcing to output of CMIP models. All explaining text has been removed, apart from where it involves the CMIP data and the changes required. For any explenation on the remaining code, see the previous notebook.

However, do have a look again at the weir at Newby Bridge. How would this place look under the different climate scenarios? Lets find out!

![image](https://upload.wikimedia.org/wikipedia/commons/7/76/Weirs_on_the_River_Leven_at_Newby_Bridge_-_geograph.org.uk_-_5455774.jpg)

*Weirs on the River Leven at Newby Bridge by G Laird*

#### Starting up

In [1]:
# general python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

#niceties
from rich import print

In [2]:
# general eWaterCycle
import ewatercycle
import ewatercycle.models
import ewatercycle.forcing

# Choose region and time period: 
Of course we need to change the time period to some future period covered by CMIP

In [3]:
camelsgb_ids = ["camelsgb_47001", "camelsgb_73010"]

In [4]:
historical_start_date = "1990-08-01T00:00:00Z"
historical_end_date = "2000-08-31T00:00:00Z"

future_start_date = "2025-08-01T00:00:00Z" #the future starts in a year from now...
future_end_date = "2035-08-31T00:00:00Z"

In [5]:
scenarios = ["ssp126","ssp245","ssp585"]

# CMIP dataset information
The CMIP project is immense... It requires either intimate knowledge of how the climate community works and stores their data, or someone in that community a phonecall away, to make sure you know which data you really want.

In eWaterCycle we use [ESMValTool](https://esmvaltool.org/) to generate forcing, which gives us the oppertunity to on the fly download CMIP (and other climate) data from available ESGF nodes around the world, provided we know exactly "how to ask". As eWaterCycle user you have to provide a dictionary like the one below as input to the ```ewatercycle.forcing.generate()``` to get the data you want.

In [13]:
cmip_dataset_future =  {
    'project': 'CMIP6',
    'activity': 'ScenarioMIP',
    'exp': 'ssp585',
    'mip': 'day',
    'dataset': 'MPI-ESM1-2-HR',
    'ensemble': 'r1i1p1f1',
    'institute': 'DKRZ',
    'grid': 'gn'
}

cmip_dataset_historical =  {
    "dataset": "MPI-ESM1-2-LR",
    "project": "CMIP6",
    "grid": "gn",
    "exp": "historical",
    "ensemble": "r3i1p1f1"
}

#### Set up paths
Below we change the forcing path from ERA5 to CMIP

In [7]:
forcing_path_caravan = {}
forcing_path_CMIP_historical = {}
forcing_path_CMIP_scenarios = {}

for camelsgb_id in camelsgb_ids:
    forcing_path_caravan[camelsgb_id] = Path.home() / "forcing" / camelsgb_id / "caravan"
    forcing_path_caravan[camelsgb_id].mkdir(exist_ok=True, parents=True)
    
    prepared_forcing_path_caravan_central = Path("/data/eurocsdms-data/forcing/camelsgb_73010/caravan")

    forcing_path_CMIP = Path.home() / "forcing" / camelsgb_id / "CMIP"
    forcing_path_CMIP.mkdir(exist_ok=True)
    
    forcing_path_CMIP_historical[camelsgb_id] = Path.home() / "forcing" / camelsgb_id / "CMIP" / "historical"
    forcing_path_CMIP_historical[camelsgb_id].mkdir(exist_ok=True)
    
    forcing_path_CMIP_scenarios[camelsgb_id] = {}
    for scenario in scenarios:
        forcing_path_CMIP_scenarios[camelsgb_id][scenario] = Path.home() / "forcing" / camelsgb_id / "CMIP" / scenario
        forcing_path_CMIP_scenarios[camelsgb_id][scenario].mkdir(exist_ok=True)
    
#prepared_forcing_path_ERA5_central = Path("/data/eurocsdms-data/forcing/camelsgb_73010/ERA5")

# Generate CMIP Forcing
In contrast to the first notebook, here we generate the CMIP data (since that is what this notebook is about). Note that the only thing that changes from the ERA5 forcing of the previous notebook is that instead of ```dataset = "ERA5"``` we now pass the dictionary with CMIP information: ```dataset = cmip_dataset```

In [9]:
# option one: get camels GB data for shapefile and for observation data.
camelsgb_forcing = {}
for camelsgb_id in camelsgb_ids:
    camelsgb_forcing[camelsgb_id] = ewatercycle.forcing.sources['CaravanForcing'].generate(
        start_time=historical_start_date,
        end_time=historical_end_date,
        directory=forcing_path_caravan[camelsgb_id],
        basin_id=camelsgb_id,
    )

In [10]:
print(camelsgb_forcing[camelsgb_id])

CaravanForcing(
    start_time='1990-08-01T00:00:00Z',
    end_time='2000-08-31T00:00:00Z',
    directory=PosixPath('/home/rhut/forcing/camelsgb_73010/caravan'),
    shape=PosixPath('/home/rhut/forcing/camelsgb_73010/caravan/camelsgb_73010.shp'),
    filenames={
        'Q': 'camelsgb_73010_1990-08-01_2000-08-31_Q.nc',
        'tasmax': 'camelsgb_73010_1990-08-01_2000-08-31_tasmax.nc',
        'tas': 'camelsgb_73010_1990-08-01_2000-08-31_tas.nc',
        'evspsblpot': 'camelsgb_73010_1990-08-01_2000-08-31_evspsblpot.nc',
        'pr': 'camelsgb_73010_1990-08-01_2000-08-31_pr.nc',
        'tasmin': 'camelsgb_73010_1990-08-01_2000-08-31_tasmin.nc'
    }
)

In [14]:
# option one: generate forcing:
CMIP_historical_forcing = {}
for camelsgb_id in camelsgb_ids:
    CMIP_historical_forcing[camelsgb_id] = ewatercycle.forcing.sources["LumpedMakkinkForcing"].generate(
       dataset=cmip_dataset_historical,
       start_time=historical_start_date,
       end_time=historical_end_date,
       directory=forcing_path_CMIP_historical[camelsgb_id],
       shape=camelsgb_forcing[camelsgb_id].shape,
    )

# # option two: load data that you or someone else generated previously
# #   this is needed because ERA5 forcing data is stored deep in a sub-directory
# load_location = prepared_forcing_path_ERA5_central / "work" / "diagnostic" / "script" 
# ERA5_forcing = ewatercycle.forcing.sources["LumpedMakkinkForcing"].load(directory=load_location)

In [18]:
CMIP_future_forcing = {}
for camelsgb_id in camelsgb_ids:
    CMIP_future_forcing[camelsgb_id] = {}
    for scenario in scenarios:
        cmip_dataset_future['exp'] = scenario
        CMIP_future_forcing[scenario] = {}

        CMIP_future_forcing[camelsgb_id][scenario] = ewatercycle.forcing.sources["LumpedMakkinkForcing"].generate(
           dataset=cmip_dataset_future,
           start_time=future_start_date,
           end_time=future_end_date,
           directory=forcing_path_CMIP_scenarios[camelsgb_id][scenario],
           shape=camelsgb_forcing[camelsgb_id].shape,
        )


# Load parameters from calibration
Note that we have to use historic calibration constants for future runs: we don't have observations yet!!

In [19]:
#load calibration constants
#par_0 = np.loadtxt("/data/eurocsdms-data/calibration/calibration_" + camelsgb_id + ".csv", delimiter = ",")
par_0 = {}
for camelsgb_id in camelsgb_ids:
    par_0[camelsgb_id] = np.loadtxt("/home/rhut/configFiles/calibration" + camelsgb_id + ".csv", delimiter = ",")

param_names = ["Imax", "Ce", "Sumax", "Beta", "Pmax", "Tlag", "Kf", "Ks", "FM"]
print(list(zip(param_names, np.round(par_0[camelsgb_id], decimals=3))))

FileNotFoundError: /home/rhut/configFiles/calibrationcamelsgb_47001.csv not found.

In [ ]:
#               Si,  Su, Sf, Ss, Sp
s_0 = np.array([0,  100,  0,  5,  0])

# Setting up the model ensemble using eWaterCycle DA
The only difference with the previous notebook is that here we pass CMIP instead of ERA5 forcing to the model on creation

In [ ]:
model = ewatercycle.models.HBV(forcing=CMIP_forcing)

In [ ]:
config_file, _ = model.setup(parameters=par_0, initial_storage=s_0)

In [ ]:
model.initialize(config_file)

#### Running the model

In [ ]:
Q_m = []
time = []
while model.time < model.end_time:
    model.update()
    Q_m.append(model.get_value("Q")[0])
    time.append(pd.Timestamp(model.time_as_datetime))

In [ ]:
model.finalize()

#### Process results

In [ ]:
model_output = pd.Series(data=Q_m, name="Modelled discharge", index=time)

In [ ]:
model_output.plot()
ds_forcing["Q"].plot(label="Observed discharge")
plt.legend()
plt.ylabel("Discharge (mm/d)")